In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Крок 1: Визначте моделі Pydantic для структурованих виходів

Ці моделі визначають **схему**, яку агенти будуть повертати. Використання `response_format` з Pydantic забезпечує:
- ✅ Безпечне вилучення даних за типом
- ✅ Автоматична валідація
- ✅ Відсутність помилок парсингу у відповідях у вільному тексті
- ✅ Легке умовне маршрутизування на основі полів


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Крок 2: Створення інструменту бронювання готелів

Цей інструмент буде використовувати **availability_agent**, щоб перевірити наявність номерів. Ми використовуємо декоратор `@ai_function` для:
- Перетворення функції Python у інструмент, доступний для виклику AI
- Автоматичного створення JSON-схеми для LLM
- Перевірки параметрів
- Автоматичного виклику агентами

Для цього демо:
- **Стокгольм, Сіетл, Токіо, Лондон, Амстердам** → Є номери ✅
- **Усі інші міста** → Номери відсутні ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Крок 3: Визначення функцій умов для маршрутизації

Ці функції аналізують відповідь агента і визначають, який шлях обрати в робочому процесі.

**Основний шаблон:**
1. Перевірте, чи є повідомлення `AgentExecutorResponse`
2. Розберіть структурований результат (модель Pydantic)
3. Поверніть `True` або `False` для управління маршрутизацією

Робочий процес оцінюватиме ці умови на **зв'язках**, щоб вирішити, який виконавець буде викликаний наступним.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Крок 4: Створення власного виконавця для відображення

Виконавці — це компоненти робочого процесу, які виконують трансформації або побічні ефекти. Ми використовуємо декоратор `@executor`, щоб створити власного виконавця, який відображає кінцевий результат.

**Основні поняття:**
- `@executor(id="...")` - Реєструє функцію як виконавця робочого процесу
- `WorkflowContext[Never, str]` - Типи для вхідних/вихідних даних
- `ctx.yield_output(...)` - Повертає кінцевий результат робочого процесу


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Крок 5: Завантаження змінних середовища

Налаштуйте клієнт LLM. Цей приклад працює з:
- **Моделями GitHub** (безкоштовний рівень із токеном GitHub)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Крок 6: Створення AI-агентів зі структурованими результатами

Ми створюємо **три спеціалізовані агенти**, кожен з яких обгорнутий у `AgentExecutor`:

1. **availability_agent** - Перевіряє доступність номерів у готелі за допомогою інструменту
2. **alternative_agent** - Пропонує альтернативні міста (коли немає доступних номерів)
3. **booking_agent** - Заохочує до бронювання (коли номери доступні)

**Основні характеристики:**
- `tools=[hotel_booking]` - Надає агенту доступ до інструменту
- `response_format=PydanticModel` - Забезпечує структурований JSON-вивід
- `AgentExecutor(..., id="...")` - Обгортає агента для використання у робочому процесі


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Крок 7: Створення робочого процесу з умовними зв'язками

Тепер ми використовуємо `WorkflowBuilder` для побудови графа з умовним маршрутизуванням:

**Структура робочого процесу:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Основні методи:**
- `.set_start_executor(...)` - Встановлює точку входу
- `.add_edge(from, to, condition=...)` - Додає умовний зв'язок
- `.build()` - Завершує створення робочого процесу


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Крок 8: Запустіть тестовий випадок 1 - Місто БЕЗ доступності (Париж)

Давайте протестуємо шлях **без доступності**, запросивши готелі в Парижі (який у нашій симуляції не має доступних номерів).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Крок 9: Запустіть тестовий випадок 2 - Місто З Наявністю (Стокгольм)

Тепер протестуємо шлях **наявності**, запросивши готелі в Стокгольмі (який має доступні номери в нашій симуляції).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Основні висновки та наступні кроки

### ✅ Що ви дізналися:

1. **Шаблон WorkflowBuilder**
   - Використовуйте `.set_start_executor()` для визначення точки входу
   - Використовуйте `.add_edge(from, to, condition=...)` для умовного маршрутизації
   - Викликайте `.build()` для завершення створення робочого процесу

2. **Умовна маршрутизація**
   - Функції умов перевіряють `AgentExecutorResponse`
   - Аналізуйте структуровані вихідні дані для прийняття рішень щодо маршрутизації
   - Поверніть `True`, щоб активувати зв’язок, або `False`, щоб пропустити його

3. **Інтеграція інструментів**
   - Використовуйте `@ai_function`, щоб перетворити функції Python на AI-інструменти
   - Агенти автоматично викликають інструменти, коли це необхідно
   - Інструменти повертають JSON, який агенти можуть аналізувати

4. **Структуровані вихідні дані**
   - Використовуйте моделі Pydantic для безпечного вилучення даних за типами
   - Встановіть `response_format=MyModel` під час створення агентів
   - Аналізуйте відповіді за допомогою `Model.model_validate_json()`

5. **Користувацькі виконавці**
   - Використовуйте `@executor(id="...")` для створення компонентів робочого процесу
   - Виконавці можуть трансформувати дані або виконувати побічні ефекти
   - Використовуйте `ctx.yield_output()`, щоб створювати результати робочого процесу

### 🚀 Реальні застосування:

- **Бронювання подорожей**: Перевірка доступності, пропозиція альтернатив, порівняння варіантів
- **Обслуговування клієнтів**: Маршрутизація на основі типу проблеми, настрою, пріоритету
- **Електронна комерція**: Перевірка наявності товарів, пропозиція альтернатив, обробка замовлень
- **Модерація контенту**: Маршрутизація на основі рівня токсичності, позначок користувачів
- **Процеси затвердження**: Маршрутизація на основі суми, ролі користувача, рівня ризику
- **Багатоступеневе оброблення**: Маршрутизація на основі якості даних, повноти

### 📚 Наступні кроки:

- Додайте складніші умови (кілька критеріїв)
- Реалізуйте цикли з управлінням станом робочого процесу
- Додайте підпроцеси для повторно використовуваних компонентів
- Інтегруйте з реальними API (бронювання готелів, системи інвентаризації)
- Додайте обробку помилок і резервні шляхи
- Візуалізуйте робочі процеси за допомогою вбудованих інструментів візуалізації



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникли внаслідок використання цього перекладу.
